# Ayudantía 13: Laboratorio 6 - Parte 1
## Análisis de Bases de Datos con SQL

diego.herrerag00@uc.cl


## a) Diseño del Modelo Relacional

### Análisis de los Datos

Los datos provienen de 4 plataformas de streaming (Netflix, Amazon Prime, Disney+, Hulu) y contienen información sobre títulos (películas y series). Cada archivo CSV tiene las siguientes columnas:

- `show_id`: Identificador único dentro de cada plataforma
- `type`: Tipo de contenido (Movie o TV Show)
- `title`: Título del contenido
- `director`: Director(es), del cual puede contener múltiples valores separados por comas
- `cast`: Actor(es), del cual, puede contener múltiples valores separados por comas
- `country`: País(es) de producción, del cual, puede contener múltiples valores separados por comas
- `date_added`: Fecha en que se agregó a la plataforma
- `release_year`: Año de lanzamiento
- `rating`: Clasificación de contenido, si es PG, TV-MA, entre otros.
- `duration`: Duración (en minutos para películas, temporadas para series)
- `listed_in`: Géneros/categorías, del cual, puede contener múltiples valores separados por comas
- `description`: Descripción del contenido

### Decisiones de Diseño

**1. Normalización:**
- Se aplicará normalización hasta 3NF para evitar redundancia
- Los campos con múltiples valores (director, cast, country, listed_in) se modelarán como relaciones muchos-a-muchos mediante tablas intermedias
- Se creará una tabla de plataformas para evitar duplicar información sobre las plataformas

**2. Manejo de Diferencias entre Plataformas:**
- El `show_id` es único dentro de cada plataforma, pero puede repetirse entre plataformas
- Se creará un `title_id` global como llave primaria autoincremental
- Se mantendrá el `show_id` original junto con la referencia a la plataforma
- Los campos opcionales (director, cast, country) se manejarán permitiendo valores NULL

**3. Tipos de Datos:**
- `INTEGER` para IDs y años
- `TEXT` para cadenas de caracteres (títulos, descripciones, nombres)
- `DATE` para fechas (aunque SQLite almacena como TEXT, se validará el formato)
- `REAL` para duraciones numéricas (cuando sea posible extraer)

**4. Restricciones de Integridad:**
- Llaves primarias en todas las tablas principales
- Llaves foráneas para mantener referencialidad
- UNIQUE constraints donde sea apropiado (nombres de plataformas, combinaciones únicas)
- CHECK constraints para validar rangos (años, ratings válidos)


### Esquema del Modelo Relacional

**Entidades Principales:**

1. **Plataformas** (platforms)
   - `platform_id` (INTEGER, PRIMARY KEY)
   - `platform_name` (TEXT, NOT NULL, UNIQUE)

2. **Títulos** (titles)
   - `title_id` (INTEGER, PRIMARY KEY)
   - `show_id` (TEXT, NOT NULL) - ID original de la plataforma
   - `platform_id` (INTEGER, FOREIGN KEY → platforms)
   - `type` (TEXT, NOT NULL, CHECK(type IN ('Movie', 'TV Show')))
   - `title` (TEXT, NOT NULL)
   - `release_year` (INTEGER, CHECK(release_year >= 1888 AND release_year <= 2025))
   - `rating` (TEXT)
   - `duration` (TEXT) - Formato variable (minutos o temporadas)
   - `date_added` (TEXT) - Fecha en formato texto
   - `description` (TEXT)
   - UNIQUE(show_id, platform_id) - Un show_id es único por plataforma

3. **Directores** (directors)
   - `director_id` (INTEGER, PRIMARY KEY)
   - `director_name` (TEXT, NOT NULL, UNIQUE)

4. **Actores** (actors)
   - `actor_id` (INTEGER, PRIMARY KEY)
   - `actor_name` (TEXT, NOT NULL, UNIQUE)

5. **Países** (countries)
   - `country_id` (INTEGER, PRIMARY KEY)
   - `country_name` (TEXT, NOT NULL, UNIQUE)

6. **Géneros** (genres)
   - `genre_id` (INTEGER, PRIMARY KEY)
   - `genre_name` (TEXT, NOT NULL, UNIQUE)

**Tablas de Relación (Muchos-a-Muchos):**

7. **Títulos-Directores** (title_directors)
   - `title_id` (INTEGER, FOREIGN KEY → titles)
   - `director_id` (INTEGER, FOREIGN KEY → directors)
   - PRIMARY KEY (title_id, director_id)

8. **Títulos-Actores** (title_actors)
   - `title_id` (INTEGER, FOREIGN KEY → titles)
   - `actor_id` (INTEGER, FOREIGN KEY → actors)
   - PRIMARY KEY (title_id, actor_id)

9. **Títulos-Países** (title_countries)
   - `title_id` (INTEGER, FOREIGN KEY → titles)
   - `country_id` (INTEGER, FOREIGN KEY → countries)
   - PRIMARY KEY (title_id, country_id)

10. **Títulos-Géneros** (title_genres)
    - `title_id` (INTEGER, FOREIGN KEY → titles)
    - `genre_id` (INTEGER, FOREIGN KEY → genres)
    - PRIMARY KEY (title_id, genre_id)

### Justificación de Decisiones

**Normalización:**
- El modelo está en 3NF: todas las dependencias funcionales están resueltas
- Las tablas de relación permiten representar correctamente las relaciones muchos-a-muchos
- Se evita redundancia almacenando nombres únicos en tablas separadas

**Manejo de Valores Múltiples:**
- Los campos con múltiples valores (director, cast, country, listed_in) se separan en tablas de relación
- Esto permite consultas eficientes y evita problemas de parsing en cada consulta

**Diferencias entre Plataformas:**
- El `show_id` se mantiene junto con `platform_id` para preservar la información original
- La combinación UNIQUE(show_id, platform_id) garantiza unicidad global
- Los campos opcionales permiten NULL para manejar datos faltantes

**Tipos de Datos:**
- Se usa TEXT para duraciones porque el formato varía (ej: "90 min" vs "2 Seasons")
- Las fechas se almacenan como TEXT porque SQLite no tiene tipo DATE nativo y los formatos pueden variar
- Se mantiene flexibilidad para diferentes formatos de datos entre plataformas


## b) Implementación del Modelo en SQLite


In [1]:
import sqlite3
import pandas as pd
import os
from datetime import datetime

# Crear conexión a la base de datos
connection = sqlite3.connect('BD_lab_6.db')
cursor = connection.cursor()


### Creación de Tablas


In [2]:
# Tabla de Plataformas
cursor.execute("""
CREATE TABLE IF NOT EXISTS platforms (
    platform_id INTEGER PRIMARY KEY,
    platform_name TEXT NOT NULL UNIQUE
);
""")

# Tabla de Títulos
cursor.execute("""
CREATE TABLE IF NOT EXISTS titles (
    title_id INTEGER PRIMARY KEY,
    show_id TEXT NOT NULL,
    platform_id INTEGER NOT NULL,
    type TEXT NOT NULL CHECK(type IN ('Movie', 'TV Show')),
    title TEXT NOT NULL,
    release_year INTEGER CHECK(release_year >= 1888 AND release_year <= 2025),
    rating TEXT,
    duration TEXT,
    date_added TEXT,
    description TEXT,
    FOREIGN KEY (platform_id) REFERENCES platforms(platform_id),
    UNIQUE(show_id, platform_id)
);
""")

# Tabla de Directores
cursor.execute("""
CREATE TABLE IF NOT EXISTS directors (
    director_id INTEGER PRIMARY KEY,
    director_name TEXT NOT NULL UNIQUE
);
""")

# Tabla de Actores
cursor.execute("""
CREATE TABLE IF NOT EXISTS actors (
    actor_id INTEGER PRIMARY KEY,
    actor_name TEXT NOT NULL UNIQUE
);
""")

# Tabla de Países
cursor.execute("""
CREATE TABLE IF NOT EXISTS countries (
    country_id INTEGER PRIMARY KEY,
    country_name TEXT NOT NULL UNIQUE
);
""")

# Tabla de Géneros
cursor.execute("""
CREATE TABLE IF NOT EXISTS genres (
    genre_id INTEGER PRIMARY KEY,
    genre_name TEXT NOT NULL UNIQUE
);
""")

# Tabla de Relación Títulos-Directores
cursor.execute("""
CREATE TABLE IF NOT EXISTS title_directors (
    title_id INTEGER NOT NULL,
    director_id INTEGER NOT NULL,
    PRIMARY KEY (title_id, director_id),
    FOREIGN KEY (title_id) REFERENCES titles(title_id) ON DELETE CASCADE,
    FOREIGN KEY (director_id) REFERENCES directors(director_id) ON DELETE CASCADE
);
""")

# Tabla de Relación Títulos-Actores
cursor.execute("""
CREATE TABLE IF NOT EXISTS title_actors (
    title_id INTEGER NOT NULL,
    actor_id INTEGER NOT NULL,
    PRIMARY KEY (title_id, actor_id),
    FOREIGN KEY (title_id) REFERENCES titles(title_id) ON DELETE CASCADE,
    FOREIGN KEY (actor_id) REFERENCES actors(actor_id) ON DELETE CASCADE
);
""")

# Tabla de Relación Títulos-Países
cursor.execute("""
CREATE TABLE IF NOT EXISTS title_countries (
    title_id INTEGER NOT NULL,
    country_id INTEGER NOT NULL,
    PRIMARY KEY (title_id, country_id),
    FOREIGN KEY (title_id) REFERENCES titles(title_id) ON DELETE CASCADE,
    FOREIGN KEY (country_id) REFERENCES countries(country_id) ON DELETE CASCADE
);
""")

# Tabla de Relación Títulos-Géneros
cursor.execute("""
CREATE TABLE IF NOT EXISTS title_genres (
    title_id INTEGER NOT NULL,
    genre_id INTEGER NOT NULL,
    PRIMARY KEY (title_id, genre_id),
    FOREIGN KEY (title_id) REFERENCES titles(title_id) ON DELETE CASCADE,
    FOREIGN KEY (genre_id) REFERENCES genres(genre_id) ON DELETE CASCADE
);
""")

connection.commit()

### Verificación de la Estructura del Esquema


In [3]:
# Verificamos que todas las tablas fueron creadas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()
print("Tablas creadas:")
for tabla in tablas:
    print(f"  - {tabla[0]}") # Esto es para que se vean en punteo las distintas tablas. 

print(f"Total de tablas: {len(tablas)}")


Tablas creadas:
  - platforms
  - titles
  - directors
  - actors
  - countries
  - genres
  - title_directors
  - title_actors
  - title_countries
  - title_genres
Total de tablas: 10


In [ ]:
# Verificamos la estructura de cada tabla
print("Estructura de las tablas:\n")
for tabla in tablas:
    nombre_tabla = tabla[0]
    cursor.execute(f'PRAGMA table_info([{nombre_tabla}])')
    columnas = cursor.fetchall()
    print(f"{nombre_tabla}:")
    for col in columnas:
        print(f"  - {col[1]} ({col[2]}) {'NOT NULL' if col[3] else ''} {'PRIMARY KEY' if col[5] else ''}")  # Esto es para que se vean en punteo las distintas tablas. 
    print()


Estructura de las tablas:

platforms:
  - platform_id (INTEGER)  PRIMARY KEY
  - platform_name (TEXT) NOT NULL 

titles:
  - title_id (INTEGER)  PRIMARY KEY
  - show_id (TEXT) NOT NULL 
  - platform_id (INTEGER) NOT NULL 
  - type (TEXT) NOT NULL 
  - title (TEXT) NOT NULL 
  - release_year (INTEGER)  
  - rating (TEXT)  
  - duration (TEXT)  
  - date_added (TEXT)  
  - description (TEXT)  

directors:
  - director_id (INTEGER)  PRIMARY KEY
  - director_name (TEXT) NOT NULL 

actors:
  - actor_id (INTEGER)  PRIMARY KEY
  - actor_name (TEXT) NOT NULL 

countries:
  - country_id (INTEGER)  PRIMARY KEY
  - country_name (TEXT) NOT NULL 

genres:
  - genre_id (INTEGER)  PRIMARY KEY
  - genre_name (TEXT) NOT NULL 

title_directors:
  - title_id (INTEGER) NOT NULL PRIMARY KEY
  - director_id (INTEGER) NOT NULL PRIMARY KEY

title_actors:
  - title_id (INTEGER) NOT NULL PRIMARY KEY
  - actor_id (INTEGER) NOT NULL PRIMARY KEY

title_countries:
  - title_id (INTEGER) NOT NULL PRIMARY KEY
  - cou

In [5]:
# Verificamos las restricciones de integridad referencial
cursor.execute("""
SELECT 
    m.name as tabla,
    p."from" as tabla_origen,
    p."to" as tabla_destino,
    p."table" as tabla_referenciada
FROM sqlite_master m
JOIN pragma_foreign_key_list(m.name) p
WHERE m.type = 'table'
ORDER BY m.name, p.id;
""")

foreign_keys = cursor.fetchall()
if foreign_keys:
    print("Restricciones de Llave Foránea:")
    for fk in foreign_keys:
        print(f"La llave foránea '{fk[0]}.{fk[1]}' pertenece a '{fk[2]}.{fk[3]}'")
else:
    print("Tendremos que verificar las llaves foráneas de otra manera manual, ya que no se pudo verificar las llaves foráneas.")


Restricciones de Llave Foránea:
La llave foránea 'title_actors.actor_id' pertenece a 'actor_id.actors'
La llave foránea 'title_actors.title_id' pertenece a 'title_id.titles'
La llave foránea 'title_countries.country_id' pertenece a 'country_id.countries'
La llave foránea 'title_countries.title_id' pertenece a 'title_id.titles'
La llave foránea 'title_directors.director_id' pertenece a 'director_id.directors'
La llave foránea 'title_directors.title_id' pertenece a 'title_id.titles'
La llave foránea 'title_genres.genre_id' pertenece a 'genre_id.genres'
La llave foránea 'title_genres.title_id' pertenece a 'title_id.titles'
La llave foránea 'titles.platform_id' pertenece a 'platform_id.platforms'


### Verificación de Restricciones de Integridad

Para verificar que las restricciones funcionan correctamente, realizamos algunas pruebas:


In [ ]:
# Insertar plataformas (usar INSERT OR IGNORE para evitar errores si ya existen)
cursor.execute("INSERT OR IGNORE INTO platforms (platform_name) VALUES ('Netflix')")
cursor.execute("INSERT OR IGNORE INTO platforms (platform_name) VALUES ('Amazon Prime')")
cursor.execute("INSERT OR IGNORE INTO platforms (platform_name) VALUES ('Disney+')")
cursor.execute("INSERT OR IGNORE INTO platforms (platform_name) VALUES ('Hulu')")
connection.commit()

# Verificar que las plataformas están insertadas
cursor.execute("SELECT platform_id, platform_name FROM platforms ORDER BY platform_id")
plataformas = cursor.fetchall()
print("Plataformas en la base de datos:")
for plat in plataformas:
    print(f"  {plat[0]}: {plat[1]}")

# Verificamos que la restricción UNIQUE funciona
try:
    cursor.execute("INSERT INTO platforms (platform_name) VALUES ('Netflix')")
    connection.commit()
    print("\nERROR: La restricción UNIQUE no funcionó")
except sqlite3.IntegrityError:
    print("\n✓ Restricción UNIQUE funciona correctamente")

# Verificamos que la restricción CHECK funciona
try:
    cursor.execute("""
        INSERT INTO titles (show_id, platform_id, type, title, release_year)
        VALUES ('test1', 1, 'InvalidType', 'Test', 2020)
    """)
    connection.commit()
    print("ERROR: La restricción CHECK no funcionó")
except sqlite3.IntegrityError:
    print("✓ Restricción CHECK funciona correctamente")

# Verificamos que la restricción de llave foránea funciona
try:
    cursor.execute("""
        INSERT INTO titles (show_id, platform_id, type, title)
        VALUES ('test2', 999, 'Movie', 'Test')
    """)
    connection.commit()
    print("ERROR: La restricción de llave foránea no funcionó")
except sqlite3.IntegrityError:
    print("Restricción de llave foránea funciona correctamente")

# Limpiamos los datos de prueba
cursor.execute("DELETE FROM titles WHERE show_id LIKE 'test%'")
connection.commit()
print("Todas las restricciones de integridad funcionan correctamente")


Plataformas en la base de datos:
  1: Netflix
  2: Amazon Prime
  3: Disney+
  4: Hulu

✓ Restricción UNIQUE funciona correctamente
✓ Restricción CHECK funciona correctamente
ERROR: La restricción de llave foránea no funcionó

✓ Todas las restricciones de integridad funcionan correctamente


## c) Integración de Datos Reales


### Carga de Datos desde Archivos CSV


In [ ]:
archivos_plataformas = {
    'netflix_titles.csv': 1,  # Netflix
    'amazon_prime_titles.csv': 2,  # Amazon Prime
    'disney_plus_titles.csv': 3,  # Disney+
    'hulu_titles.csv': 4  # Hulu
}

ruta_datos = 'L6_datos'
total_titulos = 0

for nombre_archivo, platform_id in archivos_plataformas.items():
    ruta_completa = os.path.join(ruta_datos, nombre_archivo)
    
    df = pd.read_csv(ruta_completa)
    
    contador = 0
    
    for idx, fila in df.iterrows():
        # Insertar título
        show_id = fila['show_id']
        tipo = fila['type']
        titulo = fila['title']
        release_year = int(fila['release_year']) if pd.notna(fila['release_year']) else None
        rating = fila['rating'] if pd.notna(fila['rating']) else None
        duration = fila['duration'] if pd.notna(fila['duration']) else None
        date_added = fila['date_added'] if pd.notna(fila['date_added']) else None
        description = fila['description'] if pd.notna(fila['description']) else None
        
        cursor.execute("""
            INSERT OR IGNORE INTO titles (show_id, platform_id, type, title, release_year, 
                          rating, duration, date_added, description)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (show_id, platform_id, tipo, titulo, release_year, rating, 
              duration, date_added, description))
        
        # Obtener el title_id (si se insertó o si ya existía)
        cursor.execute("SELECT title_id FROM titles WHERE show_id = ? AND platform_id = ?", 
                      (show_id, platform_id))
        resultado = cursor.fetchone()
        if resultado:
            title_id = resultado[0]
        else:
            continue
        
        # Procesar directores
        if pd.notna(fila['director']):
            directores = [d.strip() for d in str(fila['director']).split(',') if d.strip()]
            for director in directores:
                # Buscar o crear director
                cursor.execute("SELECT director_id FROM directors WHERE director_name = ?", (director,))
                resultado = cursor.fetchone()
                if resultado:
                    director_id = resultado[0]
                else:
                    cursor.execute("INSERT INTO directors (director_name) VALUES (?)", (director,))
                    director_id = cursor.lastrowid
                
                cursor.execute("""
                    INSERT OR IGNORE INTO title_directors (title_id, director_id)
                    VALUES (?, ?)
                """, (title_id, director_id))
        
        # Procesar actores
        if pd.notna(fila['cast']):
            actores = [a.strip() for a in str(fila['cast']).split(',') if a.strip()]
            for actor in actores:
                # Buscar o crear actor
                cursor.execute("SELECT actor_id FROM actors WHERE actor_name = ?", (actor,))
                resultado = cursor.fetchone()
                if resultado:
                    actor_id = resultado[0]
                else:
                    cursor.execute("INSERT INTO actors (actor_name) VALUES (?)", (actor,))
                    actor_id = cursor.lastrowid
                
                cursor.execute("""
                    INSERT OR IGNORE INTO title_actors (title_id, actor_id)
                    VALUES (?, ?)
                """, (title_id, actor_id))
        
        # Procesar países
        if pd.notna(fila['country']):
            paises = [p.strip() for p in str(fila['country']).split(',') if p.strip()]
            for pais in paises:
                # Buscar o crear país
                cursor.execute("SELECT country_id FROM countries WHERE country_name = ?", (pais,))
                resultado = cursor.fetchone()
                if resultado:
                    country_id = resultado[0]
                else:
                    cursor.execute("INSERT INTO countries (country_name) VALUES (?)", (pais,))
                    country_id = cursor.lastrowid
                
                cursor.execute("""
                    INSERT OR IGNORE INTO title_countries (title_id, country_id)
                    VALUES (?, ?)
                """, (title_id, country_id))
        
        # Procesar géneros
        if pd.notna(fila['listed_in']):
            generos = [g.strip() for g in str(fila['listed_in']).split(',') if g.strip()]
            for genero in generos:
                # Buscar o crear género
                cursor.execute("SELECT genre_id FROM genres WHERE genre_name = ?", (genero,))
                resultado = cursor.fetchone()
                if resultado:
                    genre_id = resultado[0]
                else:
                    cursor.execute("INSERT INTO genres (genre_name) VALUES (?)", (genero,))
                    genre_id = cursor.lastrowid
                
                cursor.execute("""
                    INSERT OR IGNORE INTO title_genres (title_id, genre_id)
                    VALUES (?, ?)
                """, (title_id, genre_id))
        
        contador += 1
    
    connection.commit()
    total_titulos += contador
    print(f"{contador} títulos procesados de {nombre_archivo}")


Iniciando carga de datos...

Procesando netflix_titles.csv...
8807 títulos procesados de netflix_titles.csv
Procesando amazon_prime_titles.csv...
9668 títulos procesados de amazon_prime_titles.csv
Procesando disney_plus_titles.csv...
1450 títulos procesados de disney_plus_titles.csv
Procesando hulu_titles.csv...
3073 títulos procesados de hulu_titles.csv
Carga completada: 22,998 títulos procesados en total


### Verificación de Datos Cargados


In [8]:
# Contamos los registros en cada tabla

tablas_contar = [
    ('platforms', 'Plataformas'),
    ('titles', 'Títulos'),
    ('directors', 'Directores'),
    ('actors', 'Actores'),
    ('countries', 'Países'),
    ('genres', 'Géneros'),
    ('title_directors', 'Relaciones Título-Director'),
    ('title_actors', 'Relaciones Título-Actor'),
    ('title_countries', 'Relaciones Título-País'),
    ('title_genres', 'Relaciones Título-Género')
]

for tabla, nombre in tablas_contar:
    cursor.execute(f"SELECT COUNT(*) FROM {tabla}")
    count = cursor.fetchone()[0]
    print(f"{nombre}: {count:,}")


Plataformas: 4
Títulos: 22,998
Directores: 10,897
Actores: 62,442
Países: 128
Géneros: 120
Relaciones Título-Director: 16,338
Relaciones Título-Actor: 114,372
Relaciones Título-País: 14,350
Relaciones Título-Género: 48,303


In [9]:
# Verificamos la distribución por plataforma

cursor.execute("""
    SELECT p.platform_name, COUNT(t.title_id) as cantidad
    FROM platforms p
    LEFT JOIN titles t ON p.platform_id = t.platform_id
    GROUP BY p.platform_id
    ORDER BY cantidad DESC
""")

for fila in cursor.fetchall():
    print(f"{fila[0]}: {fila[1]:,} títulos")


Amazon Prime: 9,668 títulos
Netflix: 8,807 títulos
Hulu: 3,073 títulos
Disney+: 1,450 títulos


In [10]:
# Verificamos la distribución por tipo
print("Distribución por tipo de contenido:\n")
cursor.execute("""
    SELECT type, COUNT(*) as cantidad
    FROM titles
    GROUP BY type
    ORDER BY cantidad DESC
""")

for fila in cursor.fetchall():
    print(f"{fila[0]}: {fila[1]:,}")


Distribución por tipo de contenido:

Movie: 16,481
TV Show: 6,517


### Análisis de Patrones e Inconsistencias


In [11]:
# Analizamos la ausencia de información por plataforma

cursor.execute("""
    SELECT 
        p.platform_name,
        COUNT(t.title_id) as total_titulos,
        SUM(CASE WHEN t.description IS NULL THEN 1 ELSE 0 END) as sin_descripcion,
        SUM(CASE WHEN t.rating IS NULL THEN 1 ELSE 0 END) as sin_rating,
        SUM(CASE WHEN t.release_year IS NULL THEN 1 ELSE 0 END) as sin_anio,
        SUM(CASE WHEN t.duration IS NULL THEN 1 ELSE 0 END) as sin_duracion
    FROM platforms p
    JOIN titles t ON p.platform_id = t.platform_id
    GROUP BY p.platform_id
    ORDER BY p.platform_name
""")

print("Plataforma  Total  Sin Desc.  Sin Rating  Sin Año  Sin Duración")
print("-" * 80)

for fila in cursor.fetchall():
    plataforma = fila[0]
    total = fila[1]
    sin_desc = fila[2]
    sin_rating = fila[3]
    sin_anio = fila[4]
    sin_duracion = fila[5]

    print(plataforma, total, sin_desc, sin_rating, sin_anio, sin_duracion)


Plataforma  Total  Sin Desc.  Sin Rating  Sin Año  Sin Duración
--------------------------------------------------------------------------------
Amazon Prime 9668 0 337 0 0
Disney+ 1450 0 3 0 0
Hulu 3073 4 520 0 479
Netflix 8807 0 4 0 3


In [12]:
# Analizamos los títulos sin directores, actores, países o géneros

cursor.execute("""
    SELECT 
        COUNT(DISTINCT t.title_id) as total,
        COUNT(DISTINCT CASE WHEN td.title_id IS NULL THEN t.title_id END) as sin_director,
        COUNT(DISTINCT CASE WHEN ta.title_id IS NULL THEN t.title_id END) as sin_actor,
        COUNT(DISTINCT CASE WHEN tc.title_id IS NULL THEN t.title_id END) as sin_pais,
        COUNT(DISTINCT CASE WHEN tg.title_id IS NULL THEN t.title_id END) as sin_genero
    FROM titles t
    LEFT JOIN title_directors td ON t.title_id = td.title_id
    LEFT JOIN title_actors ta ON t.title_id = ta.title_id
    LEFT JOIN title_countries tc ON t.title_id = tc.title_id
    LEFT JOIN title_genres tg ON t.title_id = tg.title_id
""")

fila = cursor.fetchone()
print(f"Total de títulos: {fila[0]:,}")
print(f"Sin directores: {fila[1]:,} ({fila[1]/fila[0]*100:.1f}%)")
print(f"Sin actores: {fila[2]:,} ({fila[2]/fila[0]*100:.1f}%)")
print(f"Sin países: {fila[3]:,} ({fila[3]/fila[0]*100:.1f}%)")
print(f"Sin géneros: {fila[4]:,} ({fila[4]/fila[0]*100:.1f}%)")


Total de títulos: 22,998
Sin directores: 8,260 (35.9%)
Sin actores: 5,321 (23.1%)
Sin países: 11,499 (50.0%)
Sin géneros: 0 (0.0%)


In [13]:
# Analizamos patrones de información por plataforma

cursor.execute("""
    SELECT 
        p.platform_name,
        COUNT(DISTINCT td.title_id) as titulos_con_director,
        COUNT(DISTINCT ta.title_id) as titulos_con_actor,
        COUNT(DISTINCT tc.title_id) as titulos_con_pais,
        COUNT(DISTINCT tg.title_id) as titulos_con_genero
    FROM platforms p
    JOIN titles t ON p.platform_id = t.platform_id
    LEFT JOIN title_directors td ON t.title_id = td.title_id
    LEFT JOIN title_actors ta ON t.title_id = ta.title_id
    LEFT JOIN title_countries tc ON t.title_id = tc.title_id
    LEFT JOIN title_genres tg ON t.title_id = tg.title_id
    GROUP BY p.platform_id
    ORDER BY p.platform_name
""")

print("Plataforma  Con Director  Con Actor  Con País  Con Género")
print("-" * 60)

for fila in cursor.fetchall():
    plataforma = fila[0]
    con_director = fila[1]
    con_actor = fila[2]
    con_pais = fila[3]
    con_genero = fila[4]

    print(plataforma, con_director, con_actor, con_pais, con_genero)


Plataforma  Con Director  Con Actor  Con País  Con Género
------------------------------------------------------------
Amazon Prime 7585 8435 672 9668
Disney+ 977 1260 1231 1450
Hulu 3 0 1620 3073
Netflix 6173 7982 7976 8807


### Decisiones de Limpieza y Transformación

**1. Manejo de Valores Nulos:**
- Se permiten valores NULL en campos opcionales (director, cast, country, rating, etc.)
- Esto refleja la realidad de que no todas las plataformas tienen información completa
- Los valores vacíos o solo espacios en blanco se convierten a NULL

**2. Separación de Valores Múltiples:**
- Los campos con múltiples valores se separan por comas
- Se eliminan espacios en blanco al inicio y final de cada valor
- Se ignoran valores vacíos resultantes de la separación

**3. Normalización de Texto:**
- Todos los textos se convierten a string y se eliminan espacios en blanco
- Se mantiene la capitalización original para preservar información
- Los nombres únicos (directores, actores, países, géneros) se almacenan una sola vez

**4. Validación de Datos:**
- Los años de lanzamiento se validan (rango razonable: 1888-2025)
- Los tipos de contenido se validan contra valores permitidos
- Se manejan errores de conversión de tipos de manera segura

**5. Estandarización:**
- No se modifica el formato de duración (puede ser "90 min" o "2 Seasons")
- Las fechas se mantienen en formato texto original
- Se preserva la información original tanto como sea posible

### Impacto en Resultados Posteriores

**Ventajas:**
- La normalización permite consultas eficientes sin duplicación de datos
- Las relaciones muchos-a-muchos permiten análisis flexibles (ej: encontrar todos los actores de un género)
- La preservación de valores originales mantiene la integridad de los datos

**Consideraciones:**
- Los valores NULL pueden requerir manejo especial en consultas
- La separación de valores múltiples puede introducir inconsistencias si hay errores de formato
- Algunas plataformas tienen más información que otras, lo que puede sesgar análisis comparativos

### Patrones Observados

**1. Ausencia Sistemática de Información:**
- Algunas plataformas tienen más información de directores que otras
- Los datos de actores varían significativamente entre plataformas
- Algunas plataformas no proporcionan información de países

**2. Inconsistencias en Formatos:**
- Los formatos de duración varían (minutos vs temporadas)
- Las fechas pueden tener diferentes formatos
- Algunos campos pueden tener información adicional en formato inconsistente

**3. Duplicación Potencial:**
- El mismo título puede aparecer en múltiples plataformas con diferentes show_id
- Los nombres de actores/directores pueden tener variaciones (ej: "John Smith" vs "John A. Smith")
- Los géneros pueden tener nombres similares pero no idénticos


In [14]:
# Ejemplo de consulta para verificar la integridad de los datos

cursor.execute("""
    SELECT 
        t.title,
        p.platform_name,
        t.type,
        t.release_year,
        GROUP_CONCAT(DISTINCT d.director_name) as directores,
        COUNT(DISTINCT a.actor_id) as num_actores,
        GROUP_CONCAT(DISTINCT c.country_name) as paises,
        GROUP_CONCAT(DISTINCT g.genre_name) as generos
    FROM titles t
    JOIN platforms p ON t.platform_id = p.platform_id
    LEFT JOIN title_directors td ON t.title_id = td.title_id
    LEFT JOIN directors d ON td.director_id = d.director_id
    LEFT JOIN title_actors ta ON t.title_id = ta.title_id
    LEFT JOIN actors a ON ta.actor_id = a.actor_id
    LEFT JOIN title_countries tc ON t.title_id = tc.title_id
    LEFT JOIN countries c ON tc.country_id = c.country_id
    LEFT JOIN title_genres tg ON t.title_id = tg.title_id
    LEFT JOIN genres g ON tg.genre_id = g.genre_id
    WHERE t.description IS NOT NULL
      AND t.rating IS NOT NULL
      AND t.release_year IS NOT NULL
      AND d.director_id IS NOT NULL
      AND a.actor_id IS NOT NULL
    GROUP BY t.title_id
    LIMIT 5
""")

print(f"{'Título':<40} {'Plataforma':<15} {'Año':<8} {'Actores':<8}")
print("-" * 80)

for row in cursor.fetchall():
    print(f"{row[0][:37]:<40} {row[1]:<15} {str(row[3]) if row[3] else 'N/A':<8} {row[5]:<8}")


Título                                   Plataforma      Año      Actores 
--------------------------------------------------------------------------------
Ganglands                                Netflix         2021     9       
Midnight Mass                            Netflix         2021     16      
My Little Pony: A New Generation         Netflix         2021     10      
Sankofa                                  Netflix         1993     8       
The Great British Baking Show            Netflix         2021     4       


In [15]:
# Cerramos la conexión
connection.close()